Este código gera o gráfico da margem de erro, onde o experimento foi executado por 100 vezes

In [ ]:
import matplotlib.pyplot as plt
from itertools import product
import seaborn as sns 
import pandas as pd
import numpy as np
import glob

sns.set_theme(style='darkgrid')
tamanho_fonte = 25
lista = []
try:
    niid_iid = ['IID', 'NIID']
    ataques = ['ALTERNA_INICIO', 'ATACANTES', 'EMBARALHA', 'INVERTE_TREINANDO', 'INVERTE_SEM_TREINAR', 'INVERTE_CONVEGENCIA', 'ZEROS', 'RUIDO_GAUSSIANO', 'NORMAL']
    data_set = ['MNIST', 'CIFAR10']
    modelos = ['DNN', 'CNN']

    for i, j, k, l in product(niid_iid, ataques, data_set, modelos):
        file_list = glob.glob(f'TESTES/rodou_dez_vezes/{i}/LOG_EVALUATE/{j}_{k}_{l}*.csv')
        lista.append(file_list)

except Exception as e:
    print(f"Ocorreu um erro ao processar: {str(e)}")

for caminhos_arquivos in lista:
    rotulos = []
    for arquivo in caminhos_arquivos:
        try:
            arquivo = arquivo.replace('\\', '/')

            extensao = arquivo.split('.')
            caminho = extensao[0].split('/')
            base = caminho[4].split('_')
            rotulo = f'{base[-1]}'
            rotulos.append(rotulo)

            plt.figure(figsize=(9, 5))
            for i, arquivo_atual in enumerate(caminhos_arquivos):
                data = pd.read_csv(arquivo_atual, header=None)
                data.columns = ['server_round', 'cid', 'accuracy', 'loss']

                sns.lineplot(x='server_round', y='accuracy', data=data, label=f'{rotulos[i]}')

            xticks = np.arange(0,101,10)
            plt.xticks(xticks, fontsize=tamanho_fonte)
            plt.xticks(fontsize=tamanho_fonte)
            plt.yticks(fontsize=tamanho_fonte)

            plt.grid(color='k', linestyle='--', linewidth=0.5, axis='both', alpha=0.1)
            plt.legend(
                loc='best',
                fontsize=tamanho_fonte,
                ncol=1,
                title='# Round',
                title_fontsize=tamanho_fonte
            )
            plt.savefig(f'TESTES/{caminho[1]}/GRAFICOS/{base[0]}_{base[1]}_{base[2]}_accuracy.png', dpi=300)
            plt.close('all')


            plt.figure(figsize=(9, 5))
            for i, arquivo_atual in enumerate(caminhos_arquivos):
                data = pd.read_csv(arquivo_atual, header=None)
                data.columns = ['server_round', 'cid', 'accuracy', 'loss']

                sns.lineplot(x='server_round', y='loss', data=data, label=f'{rotulos[i]}')

            xticks = np.arange(0,101,10)
            plt.xticks(xticks, fontsize=tamanho_fonte)
            plt.xticks(fontsize=tamanho_fonte)
            plt.yticks(fontsize=tamanho_fonte)

            plt.grid(color='k', linestyle='--', linewidth=0.5, axis='both', alpha=0.1)
            plt.legend(
                loc='best',
                fontsize=tamanho_fonte,
                ncol=1,
                title='# Round',
                title_fontsize=tamanho_fonte
            )
            plt.savefig(f'TESTES/{caminho[1]}/GRAFICOS/{base[0]}_{base[1]}_{base[2]}_loss.png', dpi=300)
            plt.close('all')

        except Exception as e:
            print(f"Ocorreu um erro ao processar o arquivo {arquivo}: {str(e)}")




# plt.show()

Este código abaixo gera a tabela comparativa entre todas os ataques executados

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

def calcular_media(lista):
    return round(sum(lista) / len(lista), 2)

def criar_tabela(media_geral_acuracia, titulo, caminho_destino):
    fig, ax = plt.subplots()
    ax.axis('off')

    table_data = [['Modelo', 'Média de Acurácia']]
    for modelo, media_acuracia in media_geral_acuracia.items():
        table_data.append([modelo, media_acuracia])

    # Verificar se a lista table_data está vazia antes de tentar criar a tabela
    if len(table_data) > 1:
        table = ax.table(cellText=table_data, loc='center', cellLoc='center', colLabels=table_data.pop(0))
        table.auto_set_font_size(False)
        table.set_fontsize(14)
        table.scale(1.2, 1.2)

        plt.title(titulo)
        
        # Verificar se o diretório de destino existe, se não, criá-lo
        if not os.path.exists(caminho_destino):
            os.makedirs(caminho_destino)
        
        plt.savefig(f'{caminho_destino}/{titulo}_accuracy.png', dpi=300)
    else:
        print("Nenhum dado encontrado para criar a tabela.")

tamanho_fonte = 25
lista_iid_dnn = []
lista_iid_cnn = []
lista_niid_dnn = []
lista_niid_cnn = []

try:
    niid_iid = ['IID', 'NIID']
    ataques = ['ALTERNA_INICIO', 'ATACANTES', 'EMBARALHA', 'INVERTE_TREINANDO', 'INVERTE_SEM_TREINAR', 'INVERTE_CONVEGENCIA', 'ZEROS', 'RUIDO_GAUSSIANO', 'NORMAL']
    data_set = ['MNIST', 'CIFAR10']
    modelos = ['DNN', 'CNN']

    for i, j, k, l in product(niid_iid, ataques, data_set, modelos):
        file_list = glob.glob(f'TESTES/{i}/LOG_EVALUATE/{j}_{k}_{l}_*.csv')
        if i == 'IID' and  l == 'DNN':        
            lista_iid_dnn.append((j, l, file_list))
        elif i == 'IID' and l == 'CNN':
            lista_iid_cnn.append((j, l, file_list))        
        elif i == 'NIID' and l == 'DNN':
            lista_niid_dnn.append((j, l, file_list))
        elif i == 'NIID' and l == 'CNN':
            lista_niid_cnn.append((j, l, file_list))

except Exception as e:
    print(f"Ocorreu um erro ao processar: {str(e)}")

# Dicionário para armazenar médias de acurácia de cada modelo
media_acuracia_por_modelo_iid_dnn = {}
media_acuracia_por_modelo_iid_cnn = {}
media_acuracia_por_modelo_niid_dnn = {}
media_acuracia_por_modelo_niid_cnn = {}

# Processar os dados e calcular média de acurácia para cada modelo de ataque

# Processar os dados e calcular média de acurácia para cada modelo de ataque
def processar_arquivos(lista, media_acuracia_por_modelo):
    for ataque, modelo, arquivos in lista:
        chave = f'{ataque}_{modelo}'  # Usar uma chave única que leve em consideração o ataque e o modelo
        if chave not in media_acuracia_por_modelo:
            media_acuracia_por_modelo[chave] = []
        for arquivo in arquivos:
            try:
                arquivo = arquivo.replace('\\', '/')
                extensao = arquivo.split('.')
                caminho = '.'.join(extensao[:-1]).split('/')
                base = caminho[-1].split('_')
                rotulo = f'{base[-1]}'

                data = pd.read_csv(arquivo, header=None)
                data.columns = ['server_round', 'cid', 'accuracy', 'loss']
                media_round = calcular_media(data['accuracy'])
                
                media_acuracia_por_modelo[chave].append(media_round)

            except Exception as e:
                print(f"Ocorreu um erro ao processar o arquivo {arquivo}: {str(e)}")


# Processar os arquivos IID para DNN
processar_arquivos(lista_iid_dnn, media_acuracia_por_modelo_iid_dnn)

# Processar os arquivos NIID para DNN
processar_arquivos(lista_niid_dnn, media_acuracia_por_modelo_niid_dnn)

# Processar os arquivos IID para CNN
processar_arquivos(lista_iid_cnn, media_acuracia_por_modelo_iid_cnn)

# Processar os arquivos NIID para CNN
processar_arquivos(lista_niid_cnn, media_acuracia_por_modelo_niid_cnn)

# Calcular a média geral de acurácia para cada modelo e tipo
media_geral_acuracia_iid_dnn = {ataque: calcular_media(media_acuracias) for ataque, media_acuracias in media_acuracia_por_modelo_iid_dnn.items()}
media_geral_acuracia_niid_dnn = {ataque: calcular_media(media_acuracias) for ataque, media_acuracias in media_acuracia_por_modelo_niid_dnn.items()}
media_geral_acuracia_iid_cnn = {ataque: calcular_media(media_acuracias) for ataque, media_acuracias in media_acuracia_por_modelo_iid_cnn.items()}
media_geral_acuracia_niid_cnn = {ataque: calcular_media(media_acuracias) for ataque, media_acuracias in media_acuracia_por_modelo_niid_cnn.items()}

# Criar tabelas individuais para cada combinação de tipo (IID/NIID) e modelo (DNN/CNN)
criar_tabela(media_geral_acuracia_iid_dnn, 'IID - DNN', 'TESTES/IID/GRAFICOS')
criar_tabela(media_geral_acuracia_niid_dnn, 'NIID - DNN', 'TESTES/NIID/GRAFICOS')
criar_tabela(media_geral_acuracia_iid_cnn, 'IID - CNN', 'TESTES/IID/GRAFICOS')
criar_tabela(media_geral_acuracia_niid_cnn, 'NIID - CNN', 'TESTES/NIID/GRAFICOS')


Este gera a matriz confusão

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

try:
    modelos = ['DNN', 'CNN']
    niid_iid = ['IID','NIID']        
    ataques = ['ALTERNA_INICIO', 'ATACANTES', 'EMBARALHA', 'INVERTE_TREINANDO', 'INVERTE_SEM_TREINAR', 'INVERTE_CONVEGENCIA', 'ZEROS', 'RUIDO_GAUSSIANO', 'NORMAL']
    data_set = ['MNIST', 'CIFAR10']                        
    alpha_dirichlet = [0.0]
    noise_gaussiano = [0.0, 0.1]
    round_inicio = [4,8]
    per_cents_atacantes = [40]

    lista = set()
    combinacoes_unicas = set()        

    for i, j, k, l, m, n, o, p in product(niid_iid, ataques, data_set, modelos, per_cents_atacantes, alpha_dirichlet, noise_gaussiano, round_inicio):                    
        file_list = glob.glob(f'TESTES/{i}/LOG_ACERTOS/*.csv') 
        combinacao = (i, j, k, l, m, n, o, p)  
           
        if i == 'IID' and n > 0: 
                   
            continue
             
        if j != 'RUIDO_GAUSSIANO' and o > 0:        
            continue

        if (k == 'MNIST' and l == 'CNN') or (k == 'CIFAR10' and l == 'DNN'):            
            continue
        
        if combinacao not in combinacoes_unicas:                            
            combinacoes_unicas.add(combinacao)        
            lista.update(file_list)
        else:
                continue
             
except Exception as e:
    print(f"Ocorreu um erro ao processar: {str(e)}")

for arquivo in lista:
    try:
        plt.figure()  # Criar uma nova figura para cada iteração do loop
        
        arquivo = arquivo.replace('\\', '/')
        extensao = arquivo.split('.')
        caminho = '.'.join(extensao[:-1]).split('/')        
        base = caminho[-1].split('_')
       
        data = pd.read_csv(arquivo, header=None)        

        situacao = data[2].astype(int)
        prev = data[3].apply(lambda x: int(x.strip('[]')) if isinstance(x, str) else x)


        cm = confusion_matrix(situacao, prev)

        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Matriz de Confusão')
        plt.colorbar()
        classes = ['Negativo', 'Positivo']  
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes)
        plt.yticks(tick_marks, classes)

        fmt = 'd'
        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, format(cm[i, j], fmt),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")

        plt.ylabel('Classe Real')
        plt.xlabel('Classe Prevista')
        plt.tight_layout()
        arquivo_ = f'TESTES/{caminho[1]}/LOG_ACERTOS/GRAFICOS/{base[0]}_{base[1]}_{base[2]}_{base[3]}_{base[4]}_{base[5]}_{base[6]}_{base[7]}_MATRIZ_CONFUSAO.png'
        os.makedirs(os.path.dirname(arquivo_), exist_ok=True)
        plt.savefig(arquivo_, dpi=300)
        plt.close()
    except Exception as e:
        print(f"Ocorreu um erro ao processar: {str(e)}")


o código abaixo é responsável por gerar as tabelas de acurácia de todos os experimentos

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

def calcular_media(arquivos):
    return round(sum(arquivos) / len(arquivos), 2)

tamanho_fonte = 25
lista = []
combinacoes_unicas = []

try:
    modelos = ['DNN', 'CNN']
    niid_iid = ['IID','NIID']        
    ataques = ['ALTERNA_INICIO', 'ATACANTES', 'EMBARALHA', 'INVERTE_TREINANDO', 'INVERTE_SEM_TREINAR', 'INVERTE_CONVEGENCIA', 'ZEROS', 'RUIDO_GAUSSIANO', 'NORMAL']
    data_set = ['MNIST', 'CIFAR10']                        
    alpha_dirichlet = [0.0,0.5]
    noise_gaussiano = [0.0,0.1,]
    round_inicio = [4]
    per_cents_atacantes = [40]
            
   
         

    for i, j, k, l, m, n, o, p in product(niid_iid, ataques, data_set, modelos, per_cents_atacantes, alpha_dirichlet, noise_gaussiano, round_inicio):                    
        file_list = glob.glob(f'TESTES/{i}/LOG_EVALUATE/{j}_{k}_{l}_{m}_{n}_{o}*.csv') 
        combinacao = (i, j, k, l, m, n, o, p)  
            
        if i == 'IID' and n > 0:           
            continue

        if j != 'RUIDO_GAUSSIANO' and o > 0:        
            continue

        if (k == 'MNIST' and l == 'CNN') or (k == 'CIFAR10' and l == 'DNN'):            
            continue

        if combinacao not in combinacoes_unicas:                  
            combinacoes_unicas.append(combinacao)        
            lista.append(file_list)
        else:
                continue
except Exception as e:
    print(f"Ocorreu um erro ao processar: {str(e)}")

for arquivos in lista:
    rotulos = [] 
    for arquivo in arquivos:     
        try:
            arquivo = arquivo.replace('\\', '/')
            extensao = arquivo.split('.')
            caminho = '.'.join(extensao[:-1]).split('/') 
            nome_arquivo = caminho[3][:-1]             
            base = caminho[-1].split('_')
            rotulos.append(base[-1])
            
            # plt.figure(figsize=(9, 5))
            plt.figure(figsize=(10, 7))
            for i, arquivo_atual in enumerate(arquivos):
                data = pd.read_csv(arquivo_atual, header=None)
                data.columns = ['server_round', 'cid', 'accuracy', 'loss']
                media_round = data.groupby('server_round').agg({
                    'accuracy': calcular_media,
                }).reset_index()
                
                
                plt.plot(media_round['server_round'], media_round['accuracy'], label=f'{rotulos[i]}', linewidth=3)
                
                
            xticks = np.arange(0,21,2)
            plt.xticks(xticks, fontsize=tamanho_fonte)
            plt.xticks(fontsize=tamanho_fonte)
            plt.yticks(fontsize=tamanho_fonte)
            plt.ylabel('Accuracy', fontsize=tamanho_fonte)
            plt.xlabel('Round', fontsize=tamanho_fonte)
            # plt.xlabel('# Round', fontsize=tamanho_fonte, labelpad=20)
            # plt.ylabel('Loss', fontsize=tamanho_fonte, labelpad=10)
            plt.grid(color='k', linestyle='--', linewidth=0.5, axis='both', alpha=0.1)
            # plt.legend(
            #     loc='best',
            #     fontsize=tamanho_fonte,
            #     ncol=1,
            #     title='# Round',
            #     title_fontsize=tamanho_fonte
            # )
            arquivo_accuracy = f'TESTES/{caminho[1]}/GRAFICOS/{nome_arquivo}_accuracy.png'
            os.makedirs(os.path.dirname(arquivo_accuracy), exist_ok=True)
            plt.savefig(arquivo_accuracy, dpi=100)
            plt.close()

            plt.figure(figsize=(10, 7))
            for i, arquivo in enumerate(arquivos):
                data = pd.read_csv(arquivo, header=None)
                data.columns = ['server_round', 'cid', 'accuracy', 'loss']
                media_round = data.groupby('server_round').agg({
                    'loss': calcular_media,
                }).reset_index()
                
                plt.plot(media_round['server_round'], media_round['loss'], label=f'{rotulos[i]}', linewidth=3)

            xticks = np.arange(0,21,2)
            plt.xticks(xticks, fontsize=tamanho_fonte)
            plt.xticks(fontsize=tamanho_fonte)
            plt.yticks(fontsize=tamanho_fonte)
            plt.ylabel('Loss', fontsize=tamanho_fonte)
            plt.xlabel('Round', fontsize=tamanho_fonte)
            # plt.xlabel('# Round', fontsize=tamanho_fonte, labelpad=20)
            # plt.ylabel('Loss', fontsize=tamanho_fonte, labelpad=10)
            plt.grid(color='k', linestyle='--', linewidth=0.5, axis='both', alpha=0.1)
            # plt.legend(
            #     loc='best',
            #     fontsize=tamanho_fonte,
            #     ncol=1,
            #     title='# Round',
            #     title_fontsize=tamanho_fonte
            # )
            arquivo_loss = f'TESTES/{caminho[1]}/GRAFICOS/{nome_arquivo}_loss.png'
            os.makedirs(os.path.dirname(arquivo_loss), exist_ok=True)
            plt.savefig(arquivo_loss, dpi=100)
            plt.close()
        except Exception as e:
            print(f"Ocorreu um erro ao processar o arquivo {arquivo}: {str(e)}")
            a=e
       